In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/config.json
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/spiece.model
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/training_args.bin
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/tokenizer_config.json
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/model.safetensors
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/special_tokens_map.json
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/generation_config.json
/kaggle/input/dataseet/merged_cut.csv


In [2]:
!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10

In [3]:
from datasets import load_dataset, DatasetDict

# Load your dataset from CSV with utf-8 encoding
dataset = load_dataset(
    'csv',
    data_files='/kaggle/input/dataseet/merged_cut.csv',
    split='train',
    encoding='utf-8'
)

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# First, split off 10% as test set
train_val, test = dataset.train_test_split(test_size=0.1, seed=42).values()

# Then split remaining 90% into train (81%) and validation (9%)
train, validation = train_val.train_test_split(test_size=0.1, seed=42).values()

# Bundle everything into a DatasetDict
final_dataset = DatasetDict({
    'train': train,
    'validation': validation,
    'test': test
})

print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 121500
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 15000
    })
})


In [5]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import  MT5Tokenizer, MT5ForConditionalGeneration

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model"
 
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)

# Move model to CUDA
model.to(device)

print(f"Model loaded on {device}")

2025-04-15 04:09:39.764630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744690179.946304      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744690179.998554      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model loaded on cuda


In [6]:
max_source_length = 256
max_target_length = 128

def preprocess_function(examples):
    # Add prefix to source text
    prefixed_source = ["" + text for text in examples['source']]
    
    # Tokenize source
    model_inputs = tokenizer(
        prefixed_source,
        max_length=max_source_length,
        padding='max_length',
        truncation=True
    )

    # Tokenize target
    labels = tokenizer(
        examples['target'],
        max_length=max_target_length,
        padding='max_length',
        truncation=True
    )['input_ids']

    # Replace padding tokens with -100 (so they are ignored in loss computation)
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in seq] for seq in labels]
    model_inputs['labels'] = labels
    
    return model_inputs

# Apply preprocessing
# tokenized_dataset = final_dataset.map(preprocess_function, batched=True)
tokenized_dataset = final_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=final_dataset["train"].column_names  # removes 'source' and 'target'
)

print(tokenized_dataset)


Map:   0%|          | 0/121500 [00:00<?, ? examples/s]

Map:   0%|          | 0/13500 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 121500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
})


In [7]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

# Parameters
batch_size = 16
max_source_length = 256
max_target_length = 256

# Create a DataLoader for the original test set
test_data = final_dataset['test']
test_dataloader = DataLoader(test_data, batch_size=batch_size)

model.eval()

sources = []
predictions = []
references = []

# Evaluate
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Get source texts
        source_texts = batch['source']
        target_texts = batch['target']
        
        # Tokenize source
        inputs = tokenizer(
            source_texts,
            padding="max_length",
            truncation=True,
            max_length=max_source_length,
            return_tensors="pt"
        ).to(device)

        # Generate predictions
        output_ids = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=max_target_length
        )

        decoded_preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

        # Store results
        sources.extend(source_texts)
        predictions.extend(decoded_preds)
        references.extend(target_texts)


100%|██████████| 938/938 [25:42<00:00,  1.64s/it]


In [8]:
pip install evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Option 1: Using evaluate library (preferred for Hugging Face integration)
import evaluate
bleu = evaluate.load("sacrebleu")

# Compute BLEU score
results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("BLEU Score:", results['score'])


BLEU Score: 33.51616339762557


In [10]:
df = pd.DataFrame({
    "Source": sources,
    "Prediction": predictions,
    "Reference": references
})

df.to_csv("translation_results.csv", index=False, encoding='utf-8-sig')
print("Saved to translation_results.csv")


Saved to translation_results.csv
